# Data Cleaning & Standardization

## Objective
This notebook focuses on cleaning and standardizing the raw sales dataset identified during the data profiling stage.

Key goals:
- Handle missing values
- Standardize categorical columns
- Fix data types
- Remove duplicates
- Prepare a clean dataset for SQL analysis and Power BI reporting


In [1]:
import pandas as pd
import numpy as np

## Load Raw Dataset
The raw sales dataset generated in the previous step is loaded for cleaning operations.


In [2]:
df_raw = pd.read_csv("sales_raw.csv")             # Untouched source

## Create Working and Clean Copies of Data

To ensure data safety and reproducibility:
- `df_raw` preserves the original raw dataset (never modified)
- `df` is used as a working copy for step-by-step transformations
- `df_clean` represents the final cleaned dataset to be exported

In [ ]:
df = df_raw.copy()                                # Working reference
df_clean = df.copy()                              # Final cleaned dataset

## Standardize Column Names
Column names are cleaned for consistency and usability:
- Lowercase naming
- Snake_case format
- Removal of spaces and special characters


In [4]:
df_clean.columns

Index(['ORDER ID', 'Order Date', 'Customer_id', 'Gender', 'STATE', 'Region',
       'category', 'Subcategory', 'Product_Name', 'Quantity', 'unit price',
       'discount%', 'FINAL AMOUNT', 'payment mode', 'order_source'],
      dtype='object')

In [5]:
df_clean.columns = (df_clean.columns
                    .str.strip()
                    .str.lower()
                    .str.replace(" ", "_", regex = False)
                    .str.replace("%", "pct", regex = False)
                         )

In [6]:
df_clean.rename(columns={"customer_id" : "cust_id",
                         "subcategory" : "sub_category",
                        "discountpct" : "discount_pct"}, inplace = True)

## Data Type Corrections
Columns are converted to appropriate data types for accurate analysis.


In [8]:
df_clean["order_date"] = pd.to_datetime(df_clean["order_date"], errors = 'coerce')

In [9]:
numeric_cols = ["quantity", "unit_price", "discount_pct", "final_amount"]

for col in numeric_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")

## Standardizing Categorical Values
Inconsistent categorical values identified during profiling are standardized for consistency.


In [10]:
gender_mapping = {
    "male" : "Male",
    "m" : "Male",
    "f" : "Female",
    "female" : "Female",
    "other" : "Other"
}

df_clean["gender"] = df_clean["gender"].str.strip().str.lower().map(gender_mapping).fillna("Unknown")

In [11]:
payment_mapping = {
    "credit card" : "Credit Card",
    "debitcard" : "Debit Card",
    "debit_card" : "Debit Card",
    "cash on delivery" : "COD",
    "cod" : "COD",
    "upi" : "UPI"
}

df_clean["payment_mode"] = df_clean["payment_mode"].str.strip().str.lower().map(payment_mapping).fillna("Unknown")

In [12]:
state_mapping = {
    "delhi": "Delhi",
    "punjab": "Punjab",
    "maharashtra": "Maharashtra",
    "tamil nadu": "Tamil Nadu",
    "tn": "Tamil Nadu",
    "haryana": "Haryana",
    "west bengal": "West Bengal"
}

df_clean["state"] = (
    df_clean["state"]
    .str.strip()
    .str.lower()
    .str.replace("_", " ", regex=False)
    .map(state_mapping)
    .fillna("Unknown")
)

In [13]:
category_mapping = {"home & kitchen" : "Home & Kitchen",
                    "fashion" : "Fashion",
                    "beauty" : "Beauty",
                    "electronics" : "Electronics"
                   }

df_clean["category"] = df_clean["category"].str.strip().str.lower().map(category_mapping).fillna("Unknown")

In [14]:
sub_category_map = {"decor items" : "Home Decor",
                    "makeup" : "Makeup",
                    "accessories" : "Accessories",
                    "mobiles" : "Mobile",
                    "women wear" : "Women Wear",
                    "haircare" : "Hair Care",
                    "decor" : "Home Decor",
                    "appliances" : "Appliances",
                    "furniture" : "Furniture",
                    "mens_wear" : "Men Wear",
                    "skin_care" : "Skin Care",
                    "ethnic" : "Ethnic",
                    "skincare" : "Skin Care",
                    "Men Wear" : "Men Wear",
                    "Mobile Phones" : "Mobile"
                   }

df_clean["sub_category"] = df_clean["sub_category"].str.strip().str.lower().map(sub_category_map).fillna("Unknown")

## Duplicate Record Handling
Duplicate records are identified and removed to prevent double-counting in analysis.


In [15]:
df_clean.drop_duplicates(subset = "order_id" , keep = 'first' , inplace= True)

## Data Validation & Integrity Checks

Before finalizing the cleaned dataset, multiple validation checks are performed
to ensure compliance with business rules and analytical assumptions.

The following validations are applied:
- Quantity and unit price must be positive values
- Discount percentage must lie within a realistic range (0–80%)
- Order dates must fall within the analysis year (2024)
- Final transaction amount must correctly follow pricing and discount logic

These checks ensure financial accuracy and prevent invalid records from
skewing revenue, AOV, and discount-related analysis.

In [16]:
df_clean[
    (df_clean["quantity"] <= 0) | 
    (df_clean["unit_price"] <= 0)
]                                   

,order_id,order_date,cust_id,gender,state,region,category,sub_category,product_name,quantity,unit_price,discount_pct,final_amount,payment_mode,order_source


In [17]:
df_clean[
    (df_clean["discount_pct"] < 0) | 
    (df_clean["discount_pct"] > 80)
]

,order_id,order_date,cust_id,gender,state,region,category,sub_category,product_name,quantity,unit_price,discount_pct,final_amount,payment_mode,order_source


In [18]:
df_clean[
    (df_clean["order_date"].dt.year != 2024)
]

,order_id,order_date,cust_id,gender,state,region,category,sub_category,product_name,quantity,unit_price,discount_pct,final_amount,payment_mode,order_source


In [2]:
invalid_amounts = df_clean[
    df_clean["final_amount"] !=
    (df_clean["quantity"] * df_clean["unit_price"] * (1 - df_clean["discount_pct"]/100))
]

len(invalid_amounts)

NameError: name 'df_clean' is not defined

## Remove Non-Analytical Columns

The `order_source` column is not required for downstream SQL analysis or Power BI reporting.
Removing it simplifies the dataset and improves model clarity.

In [19]:
df_clean = df_clean.drop(columns=["order_source"])

## Export Clean Dataset

The finalized, cleaned dataset is exported for:
- SQL-based analysis
- Power BI dashboard development

In [20]:
df_clean.to_csv("sales_clean.csv", index=False)

In [22]:
import pandas as pd
from sqlalchemy import create_engine

# Load your CSV
df = pd.read_csv("sales_clean.csv")

# Create SQLAlchemy engine
engine = create_engine(
    "postgresql+psycopg2://postgres@localhost:5432/festive_sales_analysis",
    connect_args={"password": "Lakshay@1977"}  # pass the password here
)

# Test connection
try:
    conn = engine.connect()
    print("SQLAlchemy connection successful ✅")
    conn.close()
except Exception as e:
    print("SQLAlchemy connection failed ❌", e)

# Upload CSV to PostgreSQL
df.to_sql(
    "sales_fact",   # table name
    engine,
    if_exists="replace",  # replace table if it exists
    index=False,          # don’t write the DataFrame index
    chunksize=1000        # upload in chunks
)

print("Data uploaded successfully 🚀")


SQLAlchemy connection successful ✅
Data uploaded successfully 🚀
